# Assignment

In this assignment we will create a model for segmentation of tumor from abdominal CT images using custom loss function modifications to increase prediction sensitivity.

This assignment is part of the class **Introduction to Deep Learning for Medical Imaging** at University of California Irvine (CS190); more information can be found: https://github.com/peterchang77/dl_tutor/tree/master/cs190.

### Submission

Once complete, the following items must be submitted:

* final `*.ipynb` notebook
* final trained `*.hdf5` model file
* final compiled `*.csv` file with performance statistics

# Google Colab

The following lines of code will configure your Google Colab environment for this assignment.

### Enable GPU runtime

Use the following instructions to switch the default Colab instance into a GPU-enabled runtime:

```
Runtime > Change runtime type > Hardware accelerator > GPU
```

# Environment

### Jarvis library

In this notebook we will Jarvis, a custom Python package to facilitate data science and deep learning for healthcare. Among other things, this library will be used for low-level data management, stratification and visualization of high-dimensional medical data.

In [ ]:
# --- Install jarvis (only in Google Colab or local runtime)
% pip install jarvis-md

### faiss library

To facilitate fast kmeans clustering, we will use an efficient algorithm implemented by the Facebook AI Research team as part of the `faiss` library. In brief, `faiss` is a library for efficient similarity search and clustering of dense vectors. More information can be found here: https://github.com/facebookresearch/faiss.

In [ ]:
# --- Install faiss
% pip install faiss-cpu

### Imports

Use the following lines to import any additional needed libraries:

In [ ]:
import numpy as np, pandas as pd
from scipy import ndimage
import tensorflow as tf
from tensorflow.keras import Input, Model, models, losses, metrics, layers, optimizers
import faiss
from jarvis.train import datasets
from jarvis.utils import io
from jarvis.utils.display import imshow

# Data

The data used in this tutorial will consist of kidney tumor CT exams derived from the Kidney Tumor Segmentation Challenge (KiTS). More information about he KiTS Challenge can be found here: https://kits21.kits-challenge.org/.

In [ ]:
# --- Download dataset
datasets.download(name='ct/kits')

### Data loader

In this assignment, only the middle 2D slice of each volume will be used to promote fast model convergence. Since this small dataset fits easily into RAM memory, the following code block may be used to load these slices into a single Numpy array.

In [ ]:
def load_data(label=1, flip=True, a_min=-128, a_max=256):

    # --- Create data client
    _, _, client = datasets.prepare(name='ct/kits', keyword='3d')

    dats, lbls = [], []

    for sid, fnames, header in client.db.cursor():

        lbl, _ = io.load(fnames['lbl-crp'])
        
        if label in lbl:
            
            dat, _ = io.load(fnames['dat-crp'])
            dats.append(dat[48:49])
            lbls.append(lbl[48:49] >= label)

            if header['cohort-left'] and flip:
                dats[-1]= dats[-1][..., ::-1, :]
                lbls[-1]= lbls[-1][..., ::-1, :]

    dats = np.stack(dats, axis=0)
    lbls = np.stack(lbls, axis=0)
    
    # --- Nomralize dats
    dats = (dats - a_min) / (a_max - a_min)
    dats = dats.clip(min=0, max=1)

    return {'dat': dats, 'lbl': lbls}

Use the following cell to load all data into the `xs` dictionary:

In [ ]:
# --- Load data
xs = load_data()

# Clusters

To create useful clusters for semantic segmentation, consider the following potential features:

* pixel (voxel) value
* pixel (voxel) coordinate location
* CNN-derived features from algorithm training

The following block can be used to create a pixel-wise (voxel-wise) feature vector based on various permuatations.

In [ ]:
def create_features(x, x_weight=1, x_blur=3, coords_weight=1., backbone=None, backbone_weight=1., **kwargs):
    """
    Method to construct feature vector for clustering
    
    """
    x_ = [] 

    # --- Use features from raw data voxels
    if x_weight > 0:
        xx = x.copy()
        if x_blur > 0:
            xx[:, 0] = ndimage.gaussian_filter(xx[:, 0], sigma=(0, x_blur, x_blur, 0))
        x_.append(xx * x_weight)

    # --- Use features from coordinate location
    if coords_weight > 0:
        ij = np.meshgrid(*tuple([np.linspace(0, 1, 96) for _ in range(2)]), indexing='ij')
        ij = np.expand_dims(np.stack(ij, axis=-1), axis=0)
        ij = np.stack([ij] * x.shape[0], axis=0)
        x_.append(ij * coords_weight)

    # --- Use features from CNN-derived backbone
    if backbone is not None:
        yy = backbone.predict(x)
        x_.append(yy * backbone_weight)

    return np.concatenate(x_, axis=-1).reshape(x.size, -1)

Based on the chosen feature feature combination, perform clustering using the `faiss` library. The following method creates a total of `n_clusters` from the input data: 

In [ ]:
def create_clusters(x, n_clusters=8, **kwargs):

    # --- Create features
    x_ = create_features(x=x, **kwargs)

    # --- Apply kmeans clustering
    kmeans = faiss.Kmeans(x_.shape[-1], n_clusters)
    kmeans.train(x_.astype('float32'))
    clusters = kmeans.index.search(x_.astype('float32'), 1)[1].reshape(x.shape)

    return kmeans, clusters

**Note**: These code blocks for clustering do not need to be modified for this assignment.

# Training

### Define the backbone model

Use the following cell block to define your backbone for the semantic segmentation task:

### Define shared methods

Use the following cell block to define your shared methods for the pretraining and fine-tuning models. Consider the following shared components may be defined:

* generic method for creating algorithn inputs
* generic method for compiling model (including losses and metrics)

### Define pretrain models

Use the following cell block to start building your pretraining model using the backbone network.

### Define fine-tune models

Use the following cell block to start building your fine-tuning model using the backbone network.

### Train the model

Use the following cell block to train your deep clustering model.

# Evaluation

Based on the tutorial discussion, use the following cells to calculate model performance. The following metrics should be calculated:

* Dice score coefficient (mean, median, 25th percentile, 75th percentile)

### Performance

The following minimum performance metrics must be met for full credit:

* median Dice score coefficient: >0.80

### Results

When ready, create a `*.csv` file with your compiled **validation** cohort sensitivity and Dice score statistics. There is no need to submit training performance accuracy.

# Submission

Use the following line to save your model for submission:

In [ ]:
# --- Serialize a model
training.save('./model.hdf5')

### Canvas

Once you have completed this assignment, download the necessary files from Google Colab and your Google Drive. You will then need to submit the following items:

* final (completed) notebook: `[UCInetID]_assignment.ipynb`
* final (results) spreadsheet: `[UCInetID]_results.csv`
* final (trained) model: `[UCInetID]_model.hdf5`

**Important**: please submit all your files prefixed with your UCInetID as listed above. Your UCInetID is the part of your UCI email address that comes before `@uci.edu`. For example, Peter Anteater has an email address of panteater@uci.edu, so his notebooke file would be submitted under the name `panteater_notebook.ipynb`, his spreadshhet would be submitted under the name `panteater_results.csv` and and his model file would be submitted under the name `panteater_model.hdf5`.